First, create the model. This must match the model used in the interactive training notebook.

In [1]:
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [2]:
model.load_state_dict(torch.load('resnet18_automatic_loop.pth'))

<All keys matched successfully>

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [3]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [5]:
torch.save(model_trt.state_dict(), 'resnet18_automatic_loop_trt.pth')

Load the optimized model by executing the cell below

In [5]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('my_model_resnet18_834imgs_10epochs_trt.pth'))

<All keys matched successfully>

Create the racecar class

In [2]:
from jetracer.nvidia_racecar import NvidiaRacecar
import traitlets
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace with index of your controller

car = NvidiaRacecar()

car.throttle_gain = 0.2

car.steering_offset=0.1
car.steering = 0

display(controller)

Controller()

In [4]:
left_link = traitlets.dlink((controller.axes[0], 'value'), (car, 'steering'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[3], 'value'), (car, 'throttle'), transform=lambda x: x)

Create the camera class.

In [1]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224)

# camera.running = True


Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

# Widget for control

In [6]:
import ipywidgets.widgets as widgets
import cv2
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget

state_widget = ipywidgets.ToggleButtons(options=['stop', 'start'], description='start tractor', value='stop')
camera_widget = ipywidgets.Image(width=camera.width, height=camera.height)
x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x (left joystick)')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y (right joystick)', orientation='vertical')

display(state_widget, ipywidgets.HBox([x_slider, y_slider]), camera_widget)

ToggleButtons(description='start tractor', options=('stop', 'start'), value='stop')

Image(value=b'', height='224', width='224')

In [ ]:
from utils import preprocess
import numpy as np

import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg


STEERING_GAIN = 1
STEERING_BIAS = 0.00

car.throttle = 0

value_car_steering_widget = ipywidgets.FloatText(description='Value car steering')

data_collection_widget = ipywidgets.HBox([value_car_steering_widget])

display(data_collection_widget)

while True:
    
    if state_widget.value == "stop":
        car.throttle = 0
    
    else:
        
        car.throttle = -1
    
        image = camera.read()
        image = preprocess(image).half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        x = float(output[0])
        y = float(output[1])

        # расчет для Рулевого управления
        car_steering = (x * STEERING_GAIN + STEERING_BIAS) * -1

        x_slider.value = x
        y_slider.value = y


        car.steering = car_steering

        value_car_steering_widget.value = car_steering
        
        camera_widget.value = bgr8_to_jpeg(camera.value.copy())
    